In [24]:
import aitoolbox.context as aictx

context = aictx.Context.get()
sources = context.get_sources()
sources.set("prompt","What is the ultimate answer to life, the universe and everything?")

# Setup

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

/storage/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
TOKEN="hZmAMwYmrG7Xvxsg8KNmLxZ9TWWJvA"

In [5]:
model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)



Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:17<00:00,  8.87s/it]


# Service

In [25]:
prompt=aictx.Context.get().get_sources().get("prompt")
print(prompt)

What is the ultimate answer to life, the universe and everything?


In [26]:
in_tokens = tokenizer(prompt)
sequences = pipeline(
    prompt,
    max_length=500,
    penalty_alpha=0.8,
    top_k=20,
    return_full_text=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
resp = sequences[0]["generated_text"]
out_tokens = tokenizer(resp)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [27]:
print(resp)


As an AI language model, I do not have the ability to provide a definitive answer to this question as it is a philosophical and religious question that has been debated for centuries.


In [29]:
aictx.Context.get().get_destinations().set("resp",resp)